## Import

Obiettivo è confrontare la durata dei film con il rating, se sono correlati

Idea: come per la musica, chi vede i film se sono troppo lunghi o troppo corti ne è influenzato?

Idea: come per la musica, chi vede i film se sono troppo lunghi o troppo corti ne è influenzato

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
dataframeTitoli = pd.read_csv("./data/title_basics.tsv", sep="\t", low_memory=False)
dataframeRating = pd.read_csv("./data/title_ratings.tsv", sep="\t", low_memory=False)

In [ ]:
mergedDataframe = pd.merge(dataframeTitoli, dataframeRating, on="tconst")

In [ ]:
mergedDataframe.shape

In [ ]:
mergedDataframe.head()

In [ ]:
mergedDataframe.head()

## Data Cleaning

In [ ]:
mergedDataframe.info()

Check for null values

In [ ]:
mergedDataframe.isna().sum()

Dropping unnecessary columns

In [ ]:
mergedDataframe = mergedDataframe.drop(['isAdult', 'endYear'], axis=1)

In [ ]:
mergedDataframe.columns

Fix data types

In [ ]:
mergedDataframe.dtypes

In [ ]:
mergedDataframe['titleType'] = pd.Categorical(mergedDataframe['titleType'])

In [ ]:
mergedDataframe['titleType'].cat.categories

Problema dato da caratteri \N presenti nel dataset

In [ ]:
mergedDataframe.runtimeMinutes = pd.to_numeric(mergedDataframe.runtimeMinutes, errors ='coerce').fillna(0).astype('int')

In [ ]:
mergedDataframe.sort_values(by='runtimeMinutes', ascending=False).head()

In [ ]:
mergedDataframe.dtypes

In [ ]:
mergedDataframe.startYear = pd.to_numeric(mergedDataframe.startYear, errors ='coerce').fillna(0).astype('int')

### Modellazione

Risolvo criticità null values

In [ ]:
mergedDataframe[mergedDataframe['runtimeMinutes'] == 0].shape

Divisione tra movie e altri

In [ ]:
FilmDataframe = mergedDataframe[mergedDataframe['titleType'] == "movie"]

In [ ]:
FilmDataframe.shape

In [ ]:
film2 = FilmDataframe[(FilmDataframe['startYear'] == 2018) | (FilmDataframe['startYear'] == 2019)]

In [ ]:
film2 = film2[film2['runtimeMinutes'] != 0]

In [ ]:
film2 = film2[film2['numVotes'] > 15000]

In [ ]:
film2.shape

In [ ]:
film2.head()

In [ ]:
film2.sort_values(by='runtimeMinutes', ascending=False).head()

In [ ]:
film2.sort_values(by='runtimeMinutes').head()

In [ ]:
mergedDataframe[mergedDataframe['titleType'] == "tvSeries"].shape

In [ ]:
mergedDataframe[mergedDataframe['titleType'] == "tvSeries"].sort_values(by='startYear', ascending=False).head()

In [ ]:
mergedDataframe[(mergedDataframe['titleType'] == "tvSeries") & (mergedDataframe['startYear'] == 1994)].sort_values(by='averageRating', ascending=False).head(20)

In [ ]:
mergedDataframe[mergedDataframe['titleType'] == "tvMovie"].shape

In [ ]:
mergedDataframe[mergedDataframe['titleType'] == "tvMovie"].sort_values(by='startYear', ascending=False).head(10)

## DataViz

## Waffle - rapporto tra film e serie tv nel dataset

In [ ]:
from pywaffle import Waffle

In [ ]:
mergedDataframe[mergedDataframe['titleType'] == "movie"].shape

In [ ]:
mergedDataframe[mergedDataframe['titleType'] == "tvSeries"].shape

In [ ]:
num_movie = 241193
num_tvSeries = 65991
tot_df = num_movie + num_tvSeries

In [ ]:
perc_films = num_movie / tot_df * 100
print(perc_films)

In [ ]:
perc_tvs = num_tvSeries / tot_df * 100
print(perc_tvs)

In [ ]:
data = {'Movies': 78.52, 'TV Shows': 21.48}
fig = plt.figure(
    FigureClass=Waffle,
    rows=10,
    values=data,
    colors=("#006400", "#A9A9A9"),
    title={'label': 'Percentage of Films and TV Shows in the dataset', 'loc': 'left'},
    labels=["{0} ({1}%)".format(k, v) for k, v in data.items()],
    legend={'loc': 'lower left', 'bbox_to_anchor': (0, -0.4), 'ncol': len(data), 'framealpha': 0}
)
fig.gca().set_facecolor("#ffffff")
fig.set_facecolor("#ffffff")
plt.show()

In [ ]:
data = {'Movies': 78.52, 'TV Shows': 21.48}
fig = plt.figure(
    FigureClass=Waffle,
    rows=10,
    values=data,
    colors=("#006400", "#A9A9A9"),
    title={'label': 'Percentage of Films and TV Shows in the catalogue', 'loc': 'left'},
    icons='film',
    font_size=12,
    icon_legend=True,
    labels=["{0} ({1}%)".format(k, v) for k, v in data.items()],
    legend={'loc': 'lower left', 'bbox_to_anchor': (0, -0.4), 'ncol': len(data), 'framealpha': 0}
)
fig.gca().set_facecolor("#ffffff")
fig.set_facecolor("#ffffff")
plt.show()

In [ ]:
data = {'Movies': 78.52, 'TV Shows': 21.48}
fig = plt.figure(
    FigureClass=Waffle,
    rows=10,
    values=data,
    colors=("#006400", "#A9A9A9"),
    title={'label': 'Percentage of Films and TV Shows in the catalogue', 'loc': 'left'},
    icons='video',
    font_size=12,
    icon_legend=True,
    labels=["{0} ({1}%)".format(k, v) for k, v in data.items()],
    legend={'loc': 'lower left', 'bbox_to_anchor': (0, -0.4), 'ncol': len(data), 'framealpha': 0}
)
fig.gca().set_facecolor("#ffffff")
fig.set_facecolor("#ffffff")
plt.show()

source code: https://stackoverflow.com/questions/41400136/how-to-do-waffle-charts-in-python-square-piechart

## 1 - correlation btw years and duration of films

correlazione tra anni e durata dei film

domanda di ricerca: come per la musica, è vero che si è ridotta attenzione utenti e dunque si producono film meno lunghi rispetto a prima?

(forse dataset più adatto un archivio di film rispetto a netflix che ha molti film recenti;

inoltre all'inizio non si potevano fare film troppo lunghi per costi e tecnologie, quindi magari indagare trend più recente o spazio temporale ristretto)

In [ ]:
film_years = mergedDataframe[(mergedDataframe['titleType'] == "movie") & (mergedDataframe['runtimeMinutes'] != 0) & (mergedDataframe['runtimeMinutes'] <= 400) & (mergedDataframe['numVotes'] >= 500) & (mergedDataframe['startYear'] != 0)][['titleType', 'startYear', 'runtimeMinutes']]

In [ ]:
film_years_grouped = film_years.groupby('startYear')['startYear'].count()

In [ ]:
film_years.shape

In [ ]:
film_years_grouped.tail(20)

In [ ]:
import plotly.graph_objects as px
import plotly.express as go

x = film_years['startYear']
y = film_years['runtimeMinutes']

plot = px.Figure(data=[px.Scatter(
    x=x,
    y=y,
    mode='markers',)
])

plot.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                step="year",
                stepmode="backward"),
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
    )
)

plot.show()

#### Line chart anni e durata film ultimi 15 anni

In [ ]:
imdb_last15 = mergedDataframe[(mergedDataframe['titleType'] == "movie") & (mergedDataframe['runtimeMinutes'] != 0) & (mergedDataframe['numVotes'] >= 500) & (mergedDataframe['startYear'] > 2006)][['titleType', 'startYear', 'runtimeMinutes']]

In [ ]:
imdb_last15_grouped = imdb_last15.groupby('startYear', as_index=False)

In [ ]:
imdb_last15.groupby('startYear', as_index=False).size()

In [ ]:
mean_15 = imdb_last15_grouped.mean()

In [ ]:
mean_15

In [ ]:
mean_15.runtimeMinutes = mean_15.runtimeMinutes.round(2)
mean_15.head()

In [ ]:
sns.lineplot(data=mean_15, x='startYear', y='runtimeMinutes')

In [ ]:
sns.pointplot(data=mean_15, x='startYear', y='runtimeMinutes')

!!! non va bene solo la media 

!!! asse y parte da troppo avanti

## 2 - bar chart difference btw number of movies or tv shows during the years

differenza tra numero di film e numero di serie tv legato all'anno di uscita

In [ ]:
clean_df = mergedDataframe[(mergedDataframe['titleType'] == "movie") | (mergedDataframe['titleType'] == "tvSeries")][['titleType', 'startYear', 'runtimeMinutes', 'numVotes']]

In [ ]:
clean_df = clean_df[(clean_df['runtimeMinutes'] != 0) & (clean_df['numVotes'] >= 500)][['titleType', 'startYear', 'runtimeMinutes', 'numVotes']]

In [ ]:
#creazione nuovo dataframe
dataframe = pd.DataFrame({"startYear":[], "movie": [], "tvSeries":[]})
#ciclo for per ogni classe d'età
for item in clean_df["startYear"].unique():
  #query per identificare i record con punteggio peggiorato per la classe d'età in esame
  queryStringMovies = "startYear == "+str(item)+" and titleType == 'movie'"
  #query per identificare i record con punteggio migliorato per la classe d'età in esame
  queryStringTV = "startYear == "+str(item)+" and titleType == 'tvSeries'"

  #query
  resMovies = clean_df.query(queryStringMovies)
  resTV = clean_df.query(queryStringTV)
  #inserimento nel dataframe del numero di migliorati o peggiorati per la classe d'età in esame
  dataframe.loc[len(dataframe.index)] = [str(item), len(resMovies), len(resTV)]
#ordinamento delle classi d'età
dataframe = dataframe.sort_values("startYear")

dataframe.head()


In [ ]:
dataframe['movie'] = dataframe['movie'].astype('float')
dataframe['tvSeries'] = dataframe['tvSeries'].astype('float')

dataframe = pd.eval("delta = dataframe.movie - dataframe.tvSeries", target=dataframe)

In [ ]:
dataframe.head(1)

In [ ]:
import plotly.express as px
fig = px.line(dataframe, x="startYear", y=["movie", "tvSeries"])
fig.show()

In [ ]:
import plotly.express as px
fig = px.line(dataframe, x="startYear", y=["movie", "tvSeries"])
fig.update_xaxes(type="log")
fig.show()

In [ ]:
import plotly.express as px
fig = px.line(dataframe, x="startYear", y=["movie", "tvSeries"])
fig.update_xaxes(type="date")
fig.show()

In [ ]:
dataframe.dtypes

In [ ]:
dataframe['startYear'] = dataframe['startYear'].astype('int')

In [ ]:
dataframe.dtypes

In [ ]:
# aggiungo decade al df

dataframe.loc[dataframe['startYear'] <= 1930, 'decade'] = 1930
dataframe.loc[dataframe['startYear'].between(1930,1940), 'decade'] = 1940
dataframe.loc[dataframe['startYear'].between(1940,1950), 'decade'] = 1950
dataframe.loc[dataframe['startYear'].between(1950,1960), 'decade'] = 1960
dataframe.loc[dataframe['startYear'].between(1960,1970), 'decade'] = 1970
dataframe.loc[dataframe['startYear'].between(1970,1980), 'decade'] = 1980
dataframe.loc[dataframe['startYear'].between(1980,1990), 'decade'] = 1990
dataframe.loc[dataframe['startYear'].between(1990,2000), 'decade'] = 2000
dataframe.loc[dataframe['startYear'].between(2000,2010), 'decade'] = 2010
dataframe.loc[dataframe['startYear'].between(2010,2020), 'decade'] = 2020
dataframe.loc[dataframe['startYear'].between(2020,2030), 'decade'] = 2030

In [ ]:
dataframe['decade'] = pd.Categorical(dataframe['decade'])

In [ ]:
dataframe_10 = dataframe.loc[dataframe['startYear'] >= 2010]

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")
sns.set_style(style='white')

In [ ]:
clrs = ['grey' if (x > min(dataframe['delta'])) else 'red' for x in dataframe['delta']]
ax = sns.barplot(x='startYear', y='delta', palette=clrs, data=dataframe)
sns.despine()

ax.set(xticklabels=[])  
ax.set(title='Delta btw Films and TV shows')
ax.set(xlabel=None)
ax.tick_params(bottom=False)  # remove the ticks

In [ ]:
# dataframe_10['dPos'] = dataframe_10['delta'] >= 0
clrs = ['grey' if (x > min(dataframe_10['delta'])) else 'red' for x in dataframe_10['delta']]
ax = sns.barplot(x='startYear', y='delta', palette=clrs, data=dataframe_10)
sns.despine(trim=True)

ax.set(title='Delta btw Films and TV shows')

# IMDB

In [ ]:
avg_rating = film2['averageRating'].mean()
avg_rating = avg_rating.round(2)
avg_rating

In [ ]:
avg_minutes = film2['runtimeMinutes'].mean()
avg_minutes = avg_minutes.round(2)
avg_minutes

In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
output_notebook()

In [ ]:
from bokeh.plotting import figure, show, save, output_file
from bokeh.layouts import layout
from bokeh.models import Div, RangeSlider, Spinner,Span, Label, LabelSet, ColumnDataSource, NumeralTickFormatter, HoverTool
from bokeh.models.callbacks import CustomJS
from bokeh.models import Range1d

datasource = ColumnDataSource(film2)

p = figure(title="x", x_axis_label="Minuti", y_axis_label="Rating")
p.x_range = Range1d(film2["runtimeMinutes"].min(), film2["runtimeMinutes"].max()) 
p.y_range = Range1d(0, 10) 
tooltips = [
  ('Durata(Min)','@runtimeMinutes'),
  ('Media Voti','@averageRating'),
  ('Titolo','@primaryTitle')
]
p.circle( x="runtimeMinutes", y  = "averageRating", size=6, source=datasource)
p.add_tools(HoverTool(tooltips = tooltips))
riga_avg_rating = Span(location=avg_rating, dimension="width", line_width=3, line_color="#AAAAAA")
riga_avg_minutes = Span(location=avg_minutes, dimension="height", line_width=3, line_color="#AAAAAA")
p.add_layout(riga_avg_rating)
p.add_layout(riga_avg_minutes)
show(p)